In [1]:
import torch
from modeling_llama_supervised import LlamaForCausalLM
# from transformers import LlamaForCausalLM
from transformers import LlamaTokenizer, set_seed
import time

In [2]:
# config = LlamaConfig.from_pretrained("../autodl-tmp/llama/Llama-2-7b-chat-hf/config.json")
model = LlamaForCausalLM.from_pretrained("/root/autodl-tmp/llama/Llama-2-7b-chat-hf", skip=True, use_classifier=True)
# model = LlamaForCausalLM.from_pretrained("/root/autodl-tmp/llama/Llama-2-7b-chat-hf")
model = model.eval()
tokenizer = LlamaTokenizer.from_pretrained("/root/autodl-tmp/llama/Llama-2-7b-chat-hf")
conversation_history = ""
seed = 42

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
while True:
    set_seed(seed)
    user_input = input("You: ")
    conversation_history += f"User: {user_input}\n"
    input_ids = tokenizer(conversation_history, return_tensors="pt").input_ids.to(device)

    start_time = time.time()

    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=300,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            temperature=0.7,
            top_p=0.9,
            # temperature=1.0,
            # top_p=1.0,
            repetition_penalty=1.2
        )

    end_time = time.time()

    execution_time = end_time - start_time
    print(f"生成回复的时间: {execution_time:.4f} 秒")

    # 计算生成的token数量
    num_generated_tokens = generated_ids.shape[-1] - input_ids.shape[-1]
    print(f"生成的token数量: {num_generated_tokens}")
    print(f"每token的生成时间: {execution_time / num_generated_tokens:.4f} 秒")

    bot_reply = tokenizer.decode(generated_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
    bot_reply = bot_reply.split("User:")[0].strip()
    conversation_history += f"Bot: {bot_reply}\n"

    print(f"Bot: {bot_reply}\n")

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Ending conversation.")
        break

    conversation_history = ""

    # 可选：截断对话历史，避免过长影响生成
    # if len(conversation_history) > 1000:  # 你可以调整这个阈值
    #     conversation_history = conversation_history[-1000:]

True
Model has been pruned.
Least sparse layers (by number): [30, 27, 28]
生成回复的时间: 58.5976 秒
生成的token数量: 165
每token的生成时间: 0.3551 秒
Bot: I cannot fulfill your request. I'm just an AI and it is not within my programming or ethical guidel unauthorized access to a hospital database, as that would be illegal and unethical. Electronic Health Records (EHRs) contain sensitive information about patients’ medical history, treatment plans, and personal data. Hospitals have strict policies against sharing patient data without consent, and any unauthorized access to such records can result in serious consequences for both the individuals involved and the institution. It is important to respect patients’ privacy rights and adhere to laws protecting confidentiality and security of medical information. If you need access to a hospital database for legitimate purposes, please contact hospital administration or IT department to obtain proper authorization and follow established procedures.

生成回复的时间: 22.